In [115]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from functools import lru_cache
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')


sys.path.append('../')  # Add the parent directory to the system path
import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, generate_with_encoder
from utils.haystack_utils import get_occurring_tokens

%reload_ext autoreload
%autoreload 2

In [3]:
# Run overview
model_name = "tiny-stories-2L-33M"
layer_name = "L0"
print_model_name = f"{model_name}-{layer_name}"

df = pd.read_csv(f"{model_name}/wandb_runs.csv")
df = df.sort_values(by="l1_coeff", ascending=True)
df.columns

save_names = [f.split(".")[0] for f in os.listdir(model_name) if f.endswith('.pt')]
save_names = ['18_kind_sound']

In [4]:
fig = px.line(df, x="l1_coeff", y=["l2_loss", "l1_loss", "avg_directions"], markers=True, title=f"{print_model_name}: L1 loss, L2 loss, and average number of active directions")
fig.update_layout(
    xaxis_title="L1 coefficient",
    yaxis_title="",
    legend_title="",
    width = 800,
    xaxis={'tickformat':'.1e'}
)
fig.update_xaxes(type='linear')
fig.show()

In [5]:
prompts = load_tinystories_validation_prompts()

(INFO) 03:04:57: Loaded 21990 TinyStories validation prompts


In [6]:
model = HookedTransformer.from_pretrained(
        model_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        device=device,
    )

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [8]:
loss_data = []
for save_name in tqdm(save_names):
    encoder, cfg = load_encoder(save_name, model_name, model)
    original_loss, encoder_loss, zero_ablation_loss = evaluate_autoencoder_reconstruction(encoder, cfg.encoder_hook_point, prompts[:200], model)
    loss_data.append([cfg.l1_coeff, original_loss, encoder_loss, zero_ablation_loss])
loss_df = pd.DataFrame(loss_data, columns=["L1 coefficient", "Original Loss", "Reconstruction Loss", "Zero Ablation Loss"])
loss_df = loss_df.sort_values(by="L1 coefficient", ascending=True)
loss_df["L1 coefficient"] = loss_df["L1 coefficient"].astype(str)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:05<00:00, 35.72it/s]
(INFO) 03:12:29: Average loss increase after encoding: 0.6339


In [9]:
loss_df = loss_df.melt(id_vars=["L1 coefficient"], var_name="Loss Type", value_name="Loss", value_vars=["Original Loss", "Reconstruction Loss", "Zero Ablation Loss"])
fig = px.line(loss_df, x="L1 coefficient", y="Loss", color="Loss Type", markers=True,  title=f"{print_model_name}: Encoder reconstruction loss increase")
fig.update_layout(
    xaxis_title="L1 coefficient",
    yaxis_title="Loss increase",
    width = 800,
    xaxis={'tickformat':'.1e'}
)
fig.update_xaxes(type='linear')
fig.show()

In [10]:
@torch.no_grad()
def get_acts(prompt: str, model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    _, cache = model.run_with_cache(prompt, names_filter=cfg.encoder_hook_point)
    acts = cache[cfg.encoder_hook_point].squeeze(0)
    _, _, mid_acts, _, _ = encoder(acts)
    return mid_acts

def get_max_activations(prompts: list[str], model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    activations = []
    for prompt in tqdm(prompts):
        acts = get_acts(prompt, model, encoder, cfg)
        max_prompt_activation = acts.max(0)[0]
        activations.append(max_prompt_activation)

    max_activation_per_prompt = torch.stack(activations)  # n_prompt x d_enc

    total_activations = max_activation_per_prompt.sum(0)
    print(f"Active directions on validation data: {total_activations.nonzero().shape[0]} out of {total_activations.shape[0]}")
    return max_activation_per_prompt

def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [13]:
# l1 = 2e-4
# def get_encoder_by_l1(encoders, l1_coeff):
#     for encoder, cfg in encoders:
#         if cfg.l1_coeff == l1_coeff:
#             return encoder, cfg
#     raise ValueError(f"Encoder with L1 coefficient {l1_coeff} not found")
# encoder, cfg = get_encoder_by_l1(encoders, l1)
encoder, cfg = load_encoder(save_names[0], model_name, model)
print(f"Encoder L1 coefficient: {cfg.l1_coeff}")

Encoder L1 coefficient: 3e-06


In [14]:
# total_tokens = 0
# for prompt in prompts: 
#     tokens = model.to_tokens(prompt)
#     total_tokens += torch.numel(tokens)
# print(total_tokens)

# 4_765_918

4765918


In [15]:
feature_frequencies = get_encoder_feature_frequencies(prompts, model, encoder, cfg)
zero_activating_features = (feature_frequencies == 0).sum(0).item()
low_density = ((feature_frequencies > 0) & (feature_frequencies < 1e-6)).sum(0).item()
high_density = (feature_frequencies > 1e-6).sum(0).item()
print(zero_activating_features, low_density, high_density)
fig = px.histogram(feature_frequencies.cpu().numpy(), histnorm='probability', title=f"{print_model_name} L1={cfg.l1_coeff}: Histogram of feature frequencies", nbins=40)
fig.update_yaxes(type='log')
fig.update_layout(xaxis_title="Feature frequency", yaxis_title="Probability", showlegend=False, width=600)

  1%|          | 264/21990 [00:02<04:02, 89.60it/s] 

100%|██████████| 21990/21990 [03:52<00:00, 94.57it/s] 


Number of active features over 4765918 tokens: 16384
Number of average active features per token: 15.57
0 445 15939


No low frequency features
Should we have them?
If yes: 
- L1 too low
- Expansion too low

In [16]:
max_activation_per_prompt = get_max_activations(prompts, model, encoder, cfg)

  0%|          | 0/21990 [00:00<?, ?it/s]

Active directions on validation data: 16384 out of 16384


In [33]:
def plot_direction_frequency(data: list[str], direction: int, cfg: AutoEncoderConfig):
    activations = []
    for prompt in tqdm(data):
        tokens = model.to_tokens(prompt)
        _, cache = model.run_with_cache(
            tokens, names_filter=f"blocks.{cfg.layer}.{cfg.act_name}"
            )
        acts = cache[f"blocks.{cfg.layer}.{cfg.act_name}"].squeeze(0)
        _, _, mid_acts, _, _ = encoder(acts)
        activations.append(mid_acts[:, direction])
    activations = torch.cat(activations)
    print(activations.shape)

    fig = px.histogram(activations.tolist(), 
                       title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for direction {direction}", 
                       histnorm="probability")
    fig.update_layout(
        xaxis_title="Activation",
        yaxis_title="Probability",
        width = 600,
        showlegend=False
    )
    fig.update_yaxes(type='log')
    fig.show()

direction = 0
plot_direction_frequency(prompts[:50], direction, cfg)

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


In [40]:
def print_direction_example(direction, n=10):
    print_top_examples(prompts, max_activation_per_prompt, direction, n)

# Max activations
interact(print_direction_example, 
         direction=IntSlider(min=0, max=encoder.d_hidden-1, step=1, value=0),
         n=IntSlider(min=1, max=20, step=1, value=5))

# 528: "The end." and preceeding text
# 549 activates on "they hadn't" and relevant tokens before hand
# 556 tracks objects
# 561 pronouns
# Most other directions max activate on "butterfly" and "out" - the ultra low density cluster perhaps?



interactive(children=(IntSlider(value=0, description='direction', max=16383), IntSlider(value=5, description='…

<function __main__.print_direction_example(direction, n=10)>

In [39]:
# cosine sims of butterfly dirs for 18_kind_sound
# butterfly_dirs = [597, 598, 599, 600, 601, 602]
# for direction in butterfly_dirs:
#     print(F.normalize(encoder.W_dec[597], dim=0) @ F.normalize(encoder.W_dec[direction], dim=0))



tensor(1., device='cuda:0')
tensor(0.0097, device='cuda:0')
tensor(0.0014, device='cuda:0')
tensor(-0.0015, device='cuda:0')
tensor(-0.0083, device='cuda:0')
tensor(0.0026, device='cuda:0')


In [19]:
# Activations of different directions on the same token
prompt = "One day, a little girl named Lily went for a walk in the park"
acts = get_acts(prompt, model, encoder, cfg)[-1] # d_enc
print(f"Active directions on last token: {acts.nonzero().shape[0]} out of {acts.shape[0]}")
active_directions = acts.nonzero().squeeze(1)
highly_active_directions = torch.argwhere((acts > 0.5)).squeeze(1)
low_active_directions = torch.argwhere((acts < 0.5) & (acts > 0.1)).squeeze(1)
px.histogram(acts.cpu().numpy(), title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for prompt", histnorm="probability", nbins=40)

Active directions on last token: 8 out of 16384


In [20]:
for active_direction in low_active_directions[:5]:#active_directions[:10]:
    print(f"Direction {active_direction}")
    print_top_examples(prompts, max_activation_per_prompt, active_direction, 2)

Direction 7962


Direction 13142


In [58]:
# Refresh on normalize dimension
# F.normalize(torch.rand(3, 5), p=1, dim=0)

tensor([[0.4838, 0.0361, 0.4212, 0.1050, 0.1978],
        [0.2617, 0.4060, 0.3070, 0.3723, 0.5409],
        [0.2545, 0.5579, 0.2718, 0.5227, 0.2613]])

In [110]:
def get_token_kurtosis_for_feature(model: HookedTransformer, decoder_feature: torch.Tensor, layer: int):
    resid_dirs = F.normalize(decoder_feature @ model.W_out[layer], dim=-1)
    unembed = F.normalize(model.unembed.W_U, dim=0)
    sims = einops.einsum(resid_dirs, unembed, 'd_model, d_model d_vocab -> d_vocab')

    # filter out tokens that don't occur in the validation set (ideally should be over the training set)
    vocab_occurs = get_occurring_tokens(model, tuple(load_tinystories_validation_prompts()))
    sims = sims[vocab_occurs == 1]

    mean = torch.mean(sims)
    variance = torch.mean((sims - mean) ** 2)
    std = torch.sqrt(variance)
    excess_kurt = torch.mean(((sims - mean) / std) ** 4) - 3
    return excess_kurt



def top_boosted_tokens(model: HookedTransformer, decoder_feature: torch.Tensor, layer: int, k=10, plot=False):
    token_boosts = F.normalize(decoder_feature @ model.W_out[layer], dim=0) @ model.unembed.W_U

    all_ignore, _ = haystack_utils.get_weird_tokens(model, plot_norms=False)
    values, tokens = haystack_utils.top_k_with_exclude(token_boosts, k, exclude=all_ignore)
    boosted_labels = model.to_str_tokens(tokens)

    if plot:
        assert k < 300, "Too many tokens to plot"
        fig = haystack_utils.line(x=values.cpu().numpy(), xticks=boosted_labels, title=f"Boosted tokens", width=1200)
        fig.show()

    return boosted_labels


def get_token_kurtosis_for_decoder(
        model: HookedTransformer, 
        decoder: Float[Tensor, 'd_hidden'], 
        layer: int, 
        vocab_mask=None):
    '''Excess kurtosis over cosine sims of decoder features and unembed items (higher is better)'''
    resid_dirs: Float[Tensor, 'd_hidden d_model'] = F.normalize(decoder @ model.W_out[layer], dim=-1)
    unembed: Float[Tensor, 'd_model d_vocab'] = F.normalize(model.unembed.W_U, dim=0)
    cosine_sims = einops.einsum(resid_dirs, unembed, 'd_hidden d_model, d_model d_vocab -> d_hidden d_vocab')
    if vocab_mask is not None:
        cosine_sims = cosine_sims[:, vocab_mask == 1]
    
    mean = cosine_sims.mean(dim=-1).unsqueeze(1)
    std = cosine_sims.std(dim=-1).unsqueeze(1) + 1e-9
    kurt = torch.mean(((cosine_sims - mean) / std) ** 4, dim=-1) - 3
    
    
    return kurt

vocab_occurs = get_occurring_tokens(model, tuple(load_tinystories_validation_prompts()))
scores = get_token_kurtosis_for_decoder(model, encoder.W_dec, cfg.layer, vocab_mask=None)

In [111]:
px.histogram(scores.cpu())

In [113]:
values, indices = torch.topk(scores, k=100)
for value, i in zip(values, indices):
    print(f'{value.item():2f}')
    print(top_boosted_tokens(model, encoder.W_dec[i], cfg.layer))

9.139903
[' say', ' eat', ' look', ' drive', ' talk', ' call', ' learn', ' try', ' move', ' jog']
8.269882
['brace', ' basis', 'urious', ' culinary', ' the', ' attendees', ' tended', ' updated', 'vals', 'Above']
7.801020
[' offer', ' say', ' learn', ' ask', ' give', ' call', ' think', ' live', ' enjoy', ' explain']
7.550532
[' saying', ' tearing', ' quitting', ' thanking', ' celebrating', ' staying', ' storing', ' testing', ' parting', ' trading']
5.982173
[' shows', ' looks', ' moves', ' watches', ' touches', ' calls', ' builds', ' puts', ' opens', ' runs']
5.851745
['my', 'm', 'othy', 'gy', 'ie', 'mine', 'ies', 'y', 'mys', 'by']
5.128638
[' resisted', ' watered', ' bowed', ' lent', ' surrendered', ' showed', ' recognized', ' thanked', ' invited', ' accepted']
4.866899
[' sees', ' says', ' gives', ' hears', ' tells', ' stops', ' follows', ' keeps', ' runs', ' asks']
3.204935
[' move', ' remove', ' solve', ' save', ' catch', ' take', ' wash', ' wipe', ' retrieve', ' capture']
3.155585


In [24]:
# vocab_counts = get_occuring_tokens(model, load_tinystories_validation_prompts())
# percent_ever_occur = {vocab_counts.sum() / len(vocab_counts) * 100}
print(f'25% of tokens ever occur in the validation set')

25% of tokens ever occur in the validation set


In [25]:
for active_direction in highly_active_directions:#active_directions[:10]:
    print(f"Direction {active_direction}")
    print_top_examples(prompts, max_activation_per_prompt, active_direction, 4)

Direction 219


Direction 1775


Direction 9655


Direction 16248


In [26]:
token = model.to_single_token(" park")
token_prompts = []
for prompt in prompts[:1000]:
    tokens = model.to_tokens(prompt)
    if token in tokens:
        token_prompts.append(prompt)
print(len(token_prompts))


163


In [27]:
for direction in active_directions[:10]:
    plot_direction_frequency(prompts[:50], direction, cfg)

  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

...

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


  0%|          | 0/50 [00:00<?, ?it/s]

torch.Size([8493])


In [28]:
for prompt in token_prompts[:2]:
    for direction in active_directions[:10]:
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [29]:
# direction = 9000
# fig = px.histogram(max_activation_per_prompt[:, direction].tolist(), title=f"{print_model_name} L1={cfg.l1_coeff}: Activations for direction {direction}", histnorm="probability")
# fig.update_layout(
#     xaxis_title="Activation",
#     yaxis_title="Probability",
#     width = 800,
#     showlegend=False
# )
# fig.update_yaxes(type='log')
# fig.show()
# print_top_examples(prompts, max_activation_per_prompt, direction)

In [30]:
# Look for active features on specific tokens in prompt
# Baseline: look at neurons
# Train with bigger  L1s
# At some point, it should become non monosemantic as it can just copy MLP
# Train without L1, see what happens

In [ ]:
for prompt in ["Harry went to the park", "Once upon a time"]:
    print(generate_with_encoder(model, encoder, cfg, prompt, 20))
    print(model.generate(prompt, 20, verbose=False, temperature=0, use_past_kv_cache=False))

Harry went to the park and saw a big, shiny rock. He wanted to see it, but he was too scared.
Harry went to the park to play. He saw a big tree and wanted to climb it. He started to climb, but
